In [1]:
# Inspired by Dr. Scannell codes
# I trained the model with 10000 records, saved it, and evaluate on test dataset.

In [1]:
# Loading libraries

import nltk
import pandas as pd
import numpy as np
import csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load data
# Rename columns

df = pd.read_csv(r"/content/drive/MyDrive/NLP/CELTIC MUTATION/train.tsv", sep="\t", header=None, quoting = csv.QUOTE_NONE)
test = pd.read_csv(r"/content/drive/MyDrive/NLP/CELTIC MUTATION/test.tsv", sep="\t", header=None, quoting = csv.QUOTE_NONE)
df.rename(columns={0:"Word", 1:"Label"}, inplace=True)
test.rename(columns={0:"Word", 1:"Label"}, inplace=True)

In [4]:
# Inspecting Null values on train

df.isnull().sum()

Word     27
Label     0
dtype: int64

In [5]:
# Inspecting Null values on test

test.isnull().sum()

Word     5
Label    0
dtype: int64

In [6]:
# Removing null records

df.dropna(inplace=True)
test.dropna(inplace=True)

In [7]:
# Slicing data
df = df[:10000]

In [8]:
# Getting words with labels in the shape of List of List of Tuples

words = [[(df.loc[i, 'Word'], df.loc[i, 'Label'])] for i in df.index]
words_test = [[(test.loc[i, 'Word'], test.loc[i, 'Label'])] for i in test.index]

In [9]:
# list of words in data

vocabs = df['Word'].values.tolist()
test_vocabs = test['Word'].values.tolist()

In [10]:
# list of tags

tags = df['Label'].unique().tolist()
tags

['N', 'S', 'U', 'H', 'T']

In [11]:
# pre-processing
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 60
word2index = {w: i for i, w in enumerate(vocabs)}
tag2index = {t: i for i, t in enumerate(tags)}
onehot = [[word2index[w[0]] for w in s] for s in words]
X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(vocabs)-1)

In [12]:
# pre-processing
from tensorflow.keras.utils import to_categorical
onehot_y = [[tag2index[w[1]] for w in s] for s in words]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post")
y = to_categorical(y, num_classes=len(tags))

In [13]:
# runned on training process

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [14]:
# building model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
model = Sequential()
model.add(Embedding(input_dim=len(vocabs), output_dim=50, input_length=max_len))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(len(tags), activation="softmax")))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 50)            500000    
                                                                 
 bidirectional (Bidirectiona  (None, 60, 200)          120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 60, 5)            1005      
 ibuted)                                                         
                                                                 
Total params: 621,805
Trainable params: 621,805
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# fit
n_epochs=5
history = model.fit(X_train, y_train, batch_size=16, epochs=n_epochs, validation_split=0.1, verbose=1)

Epoch 1/5
507/507 [==============================] - 213s 403ms/step - loss: 0.0346 - accuracy: 0.9958 - val_loss: 0.0067 - val_accuracy: 0.9979
Epoch 2/5
507/507 [==============================] - 205s 405ms/step - loss: 0.0051 - accuracy: 0.9982 - val_loss: 0.0057 - val_accuracy: 0.9983
Epoch 3/5
507/507 [==============================] - 204s 402ms/step - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.0060 - val_accuracy: 0.9983
Epoch 4/5
507/507 [==============================] - 204s 403ms/step - loss: 0.0035 - accuracy: 0.9988 - val_loss: 0.0061 - val_accuracy: 0.9984
Epoch 5/5
507/507 [==============================] - 199s 393ms/step - loss: 0.0033 - accuracy: 0.9988 - val_loss: 0.0061 - val_accuracy: 0.9984


In [16]:
# accuracy on train and test (splitted from data)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Training Accuracy: 0.9989
Testing Accuracy: 0.9981


In [17]:
# save trained model

model.save(r"/content/drive/MyDrive/NLP/CELTIC MUTATION/model.h5")

In [11]:
#Loading and run on test

from tensorflow.keras.models import load_model

rnn_model = load_model(r"/content/drive/MyDrive/NLP/CELTIC MUTATION/model.h5")

In [12]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 50)            500000    
                                                                 
 bidirectional (Bidirectiona  (None, 60, 200)          120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 60, 5)            1005      
 ibuted)                                                         
                                                                 
Total params: 621,805
Trainable params: 621,805
Non-trainable params: 0
_________________________________________________________________


In [13]:
# pre-processing on test

from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 60
word2index = {w: i for i, w in enumerate(test_vocabs)}
tag2index = {t: i for i, t in enumerate(tags)}
onehot = [[word2index[w[0]] for w in s] for s in words_test]
X_test = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(test_vocabs)-1)

In [15]:
# pre-processing on test

from tensorflow.keras.utils import to_categorical
onehot_y = [[tag2index[w[1]] for w in s] for s in words_test]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post")
y_test = to_categorical(y, num_classes=len(tags))

In [17]:
# evaluate test dataset
loss, accuracy = rnn_model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Testing Accuracy: 0.9976
